<a href="https://colab.research.google.com/github/mafif21/dietin-app/blob/main/model/nasnetmobile/nasnetmobile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install import_ipynb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.7 MB/s eta 0:00:00


In [ ]:
import os
import import_ipynb
import shutil
import tensorflow as tf
import pandas as pd
import numpy as np
import pathlib
from PIL import ImageFile
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.applications import NASNetMobile
from google.colab import drive
drive.mount('/content/drive')
shutil.copyfile('/content/drive/MyDrive/Colab Notebooks/data_etl.ipynb', '/content/data_etl.ipynb')

Mounted at /content/drive


'/content/data_etl.ipynb'

In [ ]:
from data_etl import IndonesianFoodDataset

importing Jupyter notebook from data_etl.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = IndonesianFoodDataset(dataset_path='/content/drive/MyDrive/dataset/indonesian_foods_cleaned', split_size=0.7)

print(data.category_list)

ayam_bakar: 922
bakso: 737
bebek_goreng: 472
burger: 728
gado_gado: 1009
gudeg: 751
omelette: 727
rendang: 782
salad: 1542
sate: 1129
sayur_asem: 597
semur_jengkol: 550
['ayam_bakar', 'bakso', 'bebek_goreng', 'burger', 'gado_gado', 'gudeg', 'omelette', 'rendang', 'salad', 'sate', 'sayur_asem', 'semur_jengkol']


In [ ]:
data.SPLIT_SIZE = 0.8
train_set, val_set = data.load_data(target_size=(224,224),
                                    rotation_range=30,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.1,
                                    vertical_flip=True)

Found 7840 images belonging to 12 classes.
Found 1952 images belonging to 12 classes.


In [ ]:
pre_trained_model = NASNetMobile(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

for layer in pre_trained_model.layers:
  layer.trainable = False

19993432/19993432 [==============================] - 2s 0us/step


In [ ]:
pre_trained_model.summary()

Model: "NASNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 stem_conv1 (Conv2D)         (None, 111, 111, 32)         864       ['input_2[0][0]']             
                                                                                                  
 stem_bn1 (BatchNormalizati  (None, 111, 111, 32)         128       ['stem_conv1[0][0]']          
 on)                                                                                              
                                                                                                  
 activation_35 (Activation)  (None, 111, 111, 32)         0         ['stem_bn1[0][0]']       

In [ ]:
num_class = len(data.category_list)
last_layer = pre_trained_model.layers[-2]
pre_output = last_layer.output

# x = tf.keras.layers.GlobalAveragePooling2D()(pre_output)
# x = tf.keras.layers.Flatten()(pre_output)
# x = tf.keras.layers.Dropout(0.2)(x)
# x = tf.keras.layers.Dense(64,activation='relu')(x)
x = pre_trained_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(2048,activation='relu')(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(num_class, activation='softmax')(x)

model = tf.keras.Model(pre_trained_model.input, x)

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 stem_conv1 (Conv2D)         (None, 111, 111, 32)         864       ['input_3[0][0]']             
                                                                                                  
 stem_bn1 (BatchNormalizati  (None, 111, 111, 32)         128       ['stem_conv1[0][0]']          
 on)                                                                                              
                                                                                                  
 activation_376 (Activation  (None, 111, 111, 32)         0         ['stem_bn1[0][0]']      

In [ ]:
checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/nasnetmobile_checkpoint/cp2048.h5'
history_log_path = '/content/drive/MyDrive/Colab Notebooks/nasnetmobile_checkpoint/training2048.log'
callbacks = [
             tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10),
             tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path
                                                ,save_weights_only=True,
                                                save_freq='epoch',
                                                verbose=1,
                                                monitor='val_accuracy',
                                                mode='auto',
                                                save_best_only=True),
             tf.keras.callbacks.CSVLogger(history_log_path, separator=",", append=False)
             ]

In [ ]:
# if os.path.exists(checkpoint_path):
#   model.load_weights(checkpoint_path)

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.002),
              metrics = ['accuracy'])

In [ ]:
# ImageFile.LOAD_TRUNCATED_IMAGES = True

history = model.fit(
          train_set,
          validation_data=val_set,
          epochs = 50,
          callbacks = callbacks
          )

Epoch 1/50
245/245 [==============================] - ETA: 0s - loss: 2.3906 - accuracy: 0.3128
Epoch 1: val_accuracy improved from -inf to 0.56557, saving model to /content/drive/MyDrive/Colab Notebooks/nasnetmobile_checkpoint/cp2048.h5
245/245 [==============================] - 192s 675ms/step - loss: 2.3906 - accuracy: 0.3128 - val_loss: 1.3569 - val_accuracy: 0.5656
Epoch 2/50
245/245 [==============================] - ETA: 0s - loss: 1.4182 - accuracy: 0.5213
Epoch 2: val_accuracy improved from 0.56557 to 0.61834, saving model to /content/drive/MyDrive/Colab Notebooks/nasnetmobile_checkpoint/cp2048.h5
245/245 [==============================] - 162s 661ms/step - loss: 1.4182 - accuracy: 0.5213 - val_loss: 1.1147 - val_accuracy: 0.6183
Epoch 3/50
245/245 [==============================] - ETA: 0s - loss: 1.2331 - accuracy: 0.5955
Epoch 3: val_accuracy improved from 0.61834 to 0.62193, saving model to /content/drive/MyDrive/Colab Notebooks/nasnetmobile_checkpoint/cp2048.h5
245/245 [=

In [ ]:
historypd = pd.read_csv(history_log_path)
historypd = historypd.drop(labels = 'epoch', axis=1)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = [*range(len(acc))]

plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Accuracy", "Validation Accuracy"])
plt.show()

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.title('Training and validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Loss", "Validation Loss"])
plt.show()

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.show()

In [ ]:
np.save('/content/drive/MyDrive/Colab Notebooks/nasnetmobile_checkpoint/history.npy',history.history)

In [ ]:
export_dir = '/content/drive/MyDrive/Colab Notebooks/nasnetmobile/'
tf.saved_model.save(model, export_dir)

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

In [ ]:
tflite_model_file = pathlib.Path('/content/drive/MyDrive/Colab Notebooks/nasnetmobile/model.tflite')
tflite_model_file.write_bytes(tflite_model)

### Without Freezing

In [ ]:
pre_trained_model_unfreeze = NASNetMobile(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

for layer in pre_trained_model_unfreeze.layers:
  layer.trainable = False

In [ ]:
num_class = len(data.category_list)
last_layer = pre_trained_model_unfreeze.layers[-2]
pre_output = last_layer.output

# x = tf.keras.layers.GlobalAveragePooling2D()(pre_output)
# x = tf.keras.layers.Flatten()(pre_output)
# x = tf.keras.layers.Dropout(0.2)(x)
# x = tf.keras.layers.Dense(64,activation='relu')(x)
x2 = pre_trained_model_unfreeze.output
x2 = tf.keras.layers.GlobalAveragePooling2D()(x2)
x2 = tf.keras.layers.Dense(1024,activation='relu', kernel_regularizer=tf.keras.regularizers.L1())(x2)
x2 = tf.keras.layers.Dropout(0.5)(x2)
x2 = tf.keras.layers.Dense(num_class, activation='softmax')(x2)

model2 = tf.keras.Model(pre_trained_model_unfreeze.input, x2)

In [ ]:
checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/nasnetmobile_checkpoint/cp2.h5'
history_log_path = '/content/drive/MyDrive/Colab Notebooks/nasnetmobile_checkpoint/training2.log'
callbacks = [
             tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10),
             tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True,
                                                save_freq='epoch',
                                                verbose=1,
                                                monitor='val_loss',
                                                mode='auto',
                                                save_best_only=True),
             tf.keras.callbacks.CSVLogger(history_log_path, separator=",", append=False)
             ]

In [ ]:
model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics = ['accuracy'])

In [ ]:
history = model2.fit(
          train_set,
          validation_data=val_set,
          epochs = 30,
          callbacks = callbacks)

Epoch 1/30
244/244 [==============================] - ETA: 0s - loss: 34.9074 - accuracy: 0.1491
Epoch 1: val_loss improved from inf to 4.99230, saving model to /content/drive/MyDrive/Colab Notebooks/nasnetmobile_checkpoint/cp2.h5
244/244 [==============================] - 175s 658ms/step - loss: 34.9074 - accuracy: 0.1491 - val_loss: 4.9923 - val_accuracy: 0.1697
Epoch 2/30
244/244 [==============================] - ETA: 0s - loss: 4.5364 - accuracy: 0.1639
Epoch 2: val_loss improved from 4.99230 to 4.11704, saving model to /content/drive/MyDrive/Colab Notebooks/nasnetmobile_checkpoint/cp2.h5
244/244 [==============================] - 156s 639ms/step - loss: 4.5364 - accuracy: 0.1639 - val_loss: 4.1170 - val_accuracy: 0.1393
Epoch 3/30
244/244 [==============================] - ETA: 0s - loss: 4.1859 - accuracy: 0.1885
Epoch 3: val_loss improved from 4.11704 to 4.03733, saving model to /content/drive/MyDrive/Colab Notebooks/nasnetmobile_checkpoint/cp2.h5
244/244 [=====================

KeyboardInterrupt: ignored